In [ ]:
from imutils import face_utils
import imutils
import numpy as np
import collections
import dlib
import cv2
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import imread,subplot,imshow,show
from PIL import Image as im
from PIL import Image, ImageEnhance
from PIL import ImageOps
import os
from sklearn import preprocessing

n = 10000000 #set as a label


# directory = 'C:/Users/guee2/Downloads/02463/ppm/'
np.set_printoptions(threshold=sys.maxsize)
directory = 'E:/Users/guee2/Downloads/facedata/'
# load the input image from the folder and convert it into grayscale
for i in os.listdir(directory):
    files = 'E:/Users/guee2/Downloads/facedata/' + i
    for file in os.listdir(files):
        if file == "ppm":
            files2 = files + '/ppm' 
            n +=1
            for f in os.listdir(files2):
                files3 = files2 +"/" + f

        
                image = cv2.imread(files3,cv2.COLOR_BGR2GRAY)
    
    # Create a convex image
                def face_remap(shape):
                    remapped_image = cv2.convexHull(shape)
                    return remapped_image 
    
    # adjusting the brightness of the image
                def increase_brightness(image, value=100):
                    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                    h, s, v = cv2.split(hsv)

                    lim = 255 - value
                    v[v > lim] = 255
                    v[v <= lim] += value

                    final_hsv = cv2.merge((h, s, v))
                    image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
                    return image
                
                def normalization(y):
                    y = preprocessing.normalize([y])
                    return y
  
                
    # Adjusting the brightness of the image
                image = increase_brightness(image,value = 100)

#     # noise cancelling
#                 image = cv2.fastNlMeansDenoisingColored(image,None,10,10,7,21)
#                 b,g,r = cv2.split(image)           # get b,g,r
#                 image = cv2.merge([r,g,b])     # switch it to rgb

                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                out_face = np.zeros_like(gray)

    # initialize dlib's face detector (HOG-based) and then create the facial landmark predictor
                detector = dlib.get_frontal_face_detector()
                predictor = dlib.shape_predictor(r"C:\Users\guee2\Downloads\Task\shape_predictor_68_face_landmarks.dat")

    # detect faces in the grayscale image
                rects = detector(gray, 1)

    # loop over the face detections
                for (i, rect) in enumerate(rects):
  
    #Determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array
                    shape = predictor(gray,rect)
                    shape = face_utils.shape_to_np(shape)
    
    #initialize mask array
                    remapped_shape = np.zeros_like(shape)
                    feature_mask = np.zeros((gray.shape[0], gray.shape[1]))   

    # we extract the face
                    remapped_shape = face_remap(shape)
                    cv2.fillConvexPoly(feature_mask, remapped_shape[0:27], 1)
                    feature_mask = feature_mask.astype(np.bool)
                    out_face[feature_mask] = gray[feature_mask]

    # Reduce the unnecesary background
                    out_face = out_face[~np.all(out_face==0,axis=1)] #up & down
                    out_face = out_face[:,~(out_face==0).all(0)] #left & right
    
                    out_face = cv2.resize(out_face,dsize=(60,72))

#                     out_face = preprocessing.normalize(out_face)
#         out_face = cv2.cvtColor(out_face,cv2.COLOR_BGR2RGB)
#         cv2.imshow('Pic',out_face)
#         cv2.waitKey(0)
#         cv2.imwrite(r"C:\Users\guee2\Downloads\face\s"+ str(num)+ '.png',out_face)
#         num+=1




    # save the feature points in a text file
                    out_face = out_face.ravel()
                    
                    def w2file(out_face):
                        file = open(r"C:\Users\guee2\Downloads\02463\data4.txt","a")
                        for i in range(len(out_face)):
                            file.writelines(str(out_face[i]))
                            file.writelines(" ")
                        file.write(str(n))
                        file.write("\n")            
                        file.close
                    w2file(out_face)
                    

In [ ]:
def normzerounitvar_row(X):
    #X's size is 72 x 60, store 72 to r and 60 to c 
    r,c = X.shape 
    
    # to calculate the mean row by row 
    M = []
    for i in range(r): 
        row = X[i]
        M.append(statistics.mean(row)) #store all of the mean into list M with size 72 x 1
    
    #use built-in func to calc repmat and reshape to size 72x60 with fotran order
    repmat = np.matlib.repmat(M, 1, c).reshape((72,60),order="F") 
    repmat = repmat.astype(float)
    
    #substract two matrixes by using built-in function
    X = np.subtract(X,repmat)
    
    #calculate the std deviation row by row 
    N = []
    for i in range(r):
        row = X[i]
        N.append(statistics.stdev(row)) #store all the std dev in list N with size 72 x 1
        
    #Calculate repmat and reshape to size 72x60 with fotran order
    repmat2 = np.matlib.repmat(N,1,c).reshape((72,60),order="F")
    repmat2 = repmat2.astype(float)#datatype set as float
    
    #Divide two matrices with the built-in function
    Xn = np.divide(X,repmat2)
    return Xn

def normalizeR2(I):
    #in matlab 3d arr = [row, col, no_of_2d_arr]
    #in python 3d arr = [no_of_2d_arr, row, col]
    p,n = I.shape #size = 333 x 4320
    T = np.zeros([p,n]) #size = 333 x 4320
    faceH = 72
    faceW = 60

     # loop 333 times
    for i in range(p): 
        #store first row into list temp
        temp = I[i,:] 
        #reshape the first row into size = 72 x 60 with fotran order
        img = np.reshape(temp,(faceH,faceW),order="F")
        #call normalize function to process
        img = normzerounitvar_row(img)
        #reshape the size to 1 x 4320 with fortran order
        img2 = np.reshape(img,(1,4320),order="F")


        #store all the normalized pixel coordinates into matrix T
        T[i,:] = img2 #size = 333 x 4320
        img2 = np.reshape(img2,(72,60))        
        cv2.imshow('Image',img2)
        cv2.waitKey(0)
    return T
    
   

In [ ]:
import numpy as np
import cv2
import statistics 
import numpy.matlib

p = [] #list
with open(r"C:\Users\guee2\Downloads\02463\data4.txt") as file:
    for line in (file):
        line = line.strip()
        if line:
            x = line.split()
            y = []
#             for i in range (len(x)):
#                 if float(x[i]) == 10000001:
            z = x.pop()# get rid of the label
#             y = np.array([x]).reshape(72,60)
            y = np.array(x)
            y = y.astype(float)
            y = y.tolist()
            p.append(y)
out_face = np.array(p) 
normalizeR2(out_face)

In [ ]:
#MAKEGABORKERNAL.M
# [GaborReal, GaborImg] = MakeGaborKernal(GaborH, GaborW, U, V, Kmax,f,sigma )
import numpy as np
import math
def MakeGaborKernal(GaborH, GaborW, U, V, Kmax,f,sigma):
    HarfH = np.fix(GaborH/2)
    HarfW = np.fix(GaborW/2)
    GaborReal = np.zeros([GaborH,GaborW]) #71 x 71 
    GaborImg = np.zeros([GaborH,GaborW])

    Qu = math.pi * U / 8
    sqsigma = sigma * sigma
    Kv = Kmax/(f**V)
    
    postmean = math.exp(-sqsigma/2) #exponential
    for j in range(-int(HarfH), int(HarfH)+1):
        for i in range(-int(HarfW), int(HarfW)+1):
            tmp1 = math.exp(-(Kv*Kv*(j*j+i*i)/(2*sqsigma)))
            tmp2 = math.cos(Kv*math.cos(Qu)*i+Kv*math.sin(Qu)*j) - postmean
            tmp3 = math.sin(Kv*math.cos(Qu)*i+Kv*math.sin(Qu)*j)
            
            GaborReal[j+int(HarfH), i+int(HarfW)] = Kv*Kv*tmp1*tmp2/sqsigma;
            GaborImg[j+int(HarfH), i+int(HarfW)] = Kv*Kv*tmp1*tmp3/sqsigma;
    return GaborReal,GaborImg
            
#MAKEALLGABORKERNAL.M
# [ GaborReal, GaborImg ] = MakeAllGaborKernal( GaborH, GaborW, Gabor_num,Kmax,f,sigma)
import numpy as np
def MakeAllGaborKernal(GaborH, GaborW, Gabor_num,Kmax,f,sigma):
#     for unscented transform and principal angle:
    GaborReal = np.zeros([40,GaborH,GaborW]) #71 x 71 x 40
    GaborImg = np.zeros([40,GaborH,GaborW]) #71 x 71 x 40

#     for unscented transform and principal angle: 
    vnum = 5
    unum = 8
    if Gabor_num != vnum*unum:
        raise Exception("The Gabor num is 5 scales and 8 orientations!")
    for v in range(vnum):
        for u in range(unum):
            [GaborReal[v*8+u,:,:], GaborImg[v*8+u,:,:]] = MakeGaborKernal(GaborH, GaborW, u, v, Kmax,f,sigma)
    return GaborReal, GaborImg

#function for conv2 (used in gabor_conv)
from scipy.signal import convolve2d
def conv2(x,y,mode='same'):
    return np.rot90(convolve2d(np.rot90(x,2),np.rot90(y,2),mode=mode),2)

#GABOR_CONV.M
# [gabordata]=gabor_conv(img,Gabor_num,GaborReal, GaborImg)
import math
import statistics
def gabor_conv(img,Gabor_num,GaborReal, GaborImg):
    A = img.astype(float)
    B = []
    h,w = img.shape
    gabordata = np.zeros((Gabor_num,h,w))
    for j in range(Gabor_num):
        face_real = conv2(A,GaborReal[j,:,:])
        face_img =  conv2(A,GaborImg[j,:,:])
        face_mode = np.sqrt(face_real*2 + face_img*2)
        face_mode = np.reshape(face_mode,(-1),order='F')
        tmean = statistics.mean(face_mode)
        tstd = statistics.stdev(face_mode)
        face_mode = (face_mode - tmean) / tstd
        gabordata[j,:,:] = np.reshape(face_mode,(h,w),order='F')
    return gabordata

#USED TO CREATE G(:,:,1)
def Create_arr_g(x,y):
    z = np.zeros((len(x),len(y[0])))
    for i in range(len(x)):
#         print(x[i])
        z[i,:] = y*x[i] #[1:scale_h]' * ones(1,scale_w)
    return z

#USED TO CREATE G(:,:,2)
def Create_arr_g2(x,y): #g(:,:,2)
    l=[]
    for i in range(len(y)):
        l.append(x[0][i] * y[i])
    z = np.array([l,]*len(x[0]))
    return z

#NPARTITION_CELLNUM.M
# [pSE]=nPartition_cellnum(L,n,isTail):
import math
def nPartition_cellnum(L,n,isTail):
    # n is the hierarchical number
    nBlocks = n
    pSE = np.zeros((2,nBlocks))
    step = math.floor(L/nBlocks)
    
    for i in range(nBlocks):
        if i == 0 :
            pSE[0,i] = 1
        else : 
            pSE[0,i] = pSE[1,i-1] + 1
        pSE[1,i] = pSE[0,i] + step - 1
        if pSE[1,i] > L : 
            pSE[1,i] = L
            
    if isTail == 1 : 
        pSE[1,nBlocks] = L
    return pSE

#POOLING_EACHING.M
# [his]=pooling_eachimg(codes,hSE,wSE,nHBlocks,nWBlocks)
import numpy as np 
from IPython.core.display import HTML
def pooling_eachimg(codes,hSE,wSE,nHBlocks,nWBlocks):
    d,, = codes.shape
    newarr = np.zeros((42,12,10))
    r1c1 = []
    r1c2 = []
    r1c3 = []
    r2c1 = []
    r2c2 = []
    r2c3 = []
    r3c1 = []
    r3c2 = []
    r3c3 = []
    for dd in range(d):
        for hh in range(nHBlocks):
            hs = hSE[0,hh]
            he = hSE[1,hh]
            for ww in range(nWBlocks):
                ws = wSE[0,ww]
                we = wSE[1,ww]
                yy = codes[dd,int(hs-1):int(he),int(ws-1):int(we)] #size = 12x10
#                 newarr[dd,:,:] = yy
                if hh == 0 and ww == 0 :
#                     h1w1.append(newarr)
                    r1c1.append(yy)
                elif hh == 0 and ww == 1 :
#                     h1w2.append(newarr)
                    r1c2.append(yy)
                elif hh == 0 and ww == 2 :
#                     h1w3.append(newarr)
                    r1c3.append(yy)
                elif hh == 1 and ww == 0 :
#                     h2w1.append(newarr)
                    r2c1.append(yy)
                elif hh == 1 and ww == 1 :
#                     h2w2.append(newarr) 
                    r2c2.append(yy)
                elif hh == 1 and ww == 2 :
#                     h2w3.append(newarr) 
                    r2c3.append(yy)
                elif hh == 2 and ww == 0 :
#                     h3w1.append(newarr)
                    r3c1.append(yy)
                elif hh == 2 and ww == 1 :
#                     h3w2.append(newarr)
                    r3c2.append(yy)
                elif hh == 2 and ww == 2 :
#                     h3w3.append(newarr)
                    r3c3.append(yy) 

    r1c1 = np.array(r1c1)
    r1c2 = np.array(r1c2)
    r1c3 = np.array(r1c3)
    r2c1 = np.array(r2c1)
    r2c2 = np.array(r2c2)
    r2c3 = np.array(r2c3)
    r3c1 = np.array(r3c1)
    r3c2 = np.array(r3c2)
    r3c3 = np.array(r3c3)
    
    data = {'0':[r1c1,r2c1,r3c1],
            '1':[r1c2,r2c2,r3c2],
            '2':[r1c3,r2c3,r3c3]
           }
    df = pd.DataFrame(data,columns = ['0','1','2'] )
#     display(HTML(df.to_html())) #display all panda data

    return df